# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: bert-base-cased
* Evaluation approach: accuracy
* Fine-tuning dataset:  imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import Dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# foundation model (Bert-base-cased) 
model_name = "bert-base-cased"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

dataset = load_dataset("imdb")

# Tokenize the dataset
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

# Split the tokenized dataset into training and evaluation sets
#split_ratio = 0.8
#split_index = int(len(tokenized_dataset["input_ids"]) * split_ratio)

#train_dataset = {
 #   "input_ids": tokenized_dataset["input_ids"][:split_index],
  #  "attention_mask": tokenized_dataset["attention_mask"][:split_index],
   # "label": tokenized_dataset["label"][:split_index],
#}
#eval_dataset = {
 #   "input_ids": tokenized_dataset["input_ids"][split_index:],
  #  "attention_mask": tokenized_dataset["attention_mask"][split_index:],
   # "label": tokenized_dataset["label"][split_index:],
#}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 25000/25000 [00:28<00:00, 879.55 examples/s]


In [3]:
#train_dataset = CustomDataset(**train_dataset)
#eval_dataset = CustomDataset(**eval_dataset)
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))
# Define the evaluation function for the Trainer
#def compute_metrics(p):
#    return {"accuracy": (p.predictions.argmax(axis=1) == p.label_ids).mean()}
import numpy as np
!pip install scikit-learn
from sklearn.metrics import accuracy_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Trainer for foundation model evaluation
training_args_foundation = TrainingArguments(
    output_dir="./foundation_output",
    per_device_eval_batch_size=8,
    num_train_epochs=15
   
)
trainer_foundation = Trainer(
    model=model,
    args=training_args_foundation,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
foundation_results = trainer_foundation.evaluate()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [4]:
foundation_results

{'eval_loss': 0.696058452129364,
 'eval_accuracy': 0.512,
 'eval_runtime': 29.6153,
 'eval_samples_per_second': 33.766,
 'eval_steps_per_second': 4.221}

In [5]:
foundation_eval_results = trainer_foundation.evaluate()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:55<00:00, 893.94 examples/s]


In [7]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.2
)

In [8]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def compute_metrics(p):
    return {"accuracy": (p.predictions.argmax(axis=1) == p.label_ids).mean()}

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=30,)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset ,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.377562,0.842000
2,No log,0.521249,0.841000
3,No log,0.589655,0.852000
4,0.334300,0.800924,0.861000
5,0.334300,0.773384,0.853000
6,0.334300,1.048105,0.855000
7,0.334300,1.243301,0.841000
8,0.043100,1.230675,0.821000
9,0.043100,1.145726,0.851000
10,0.043100,1.213511,0.852000


TrainOutput(global_step=3750, training_loss=0.051383676455604536, metrics={'train_runtime': 3778.4339, 'train_samples_per_second': 7.94, 'train_steps_per_second': 0.992, 'total_flos': 7893331660800000.0, 'train_loss': 0.051383676455604536, 'epoch': 30.0})

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [13]:
peft_eval_results = trainer.evaluate()

In [14]:
print("Comparison of Pre-trained and PEFT Models:")
print("Pre-trained Model:", foundation_eval_results['eval_accuracy'])
print("PEFT Model:", peft_eval_results['eval_accuracy'])

Comparison of Pre-trained and PEFT Models:
Pre-trained Model: 0.512
PEFT Model: 0.853
